In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time

# Import API key
from Config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
temp = []
humidity= []
clouds = []
wind = []
lat = []
lng = []
missing_cities = []

# Create a set of random lat and lng combinations
# lats = np.linspace(90.000, -90.000, 1800)
# lngs = np.linspace(180.000, -180.000, 3600)
lats = np.random.uniform(low=-90.000, high=90.000, size=20)
lngs = np.random.uniform(low=-180.000, high=180.000, size=20)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(len(cities))
print(time.process_time())

19
2.28125


## Perform API Calls

In [3]:

# Starting URL for Weather Map API Call
counter = 0
cleaned_cities = []
city_count = len(cities)
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key
while time.process_time() > counter:
    for counter, city in enumerate(cities):    
        query = f"{url}&q={city}"
        response = requests.get(query).json()
        print(f"{counter+1} of {city_count} cities called")
        print(city)
        try:
            temp.append(response["main"]["temp"])
            humidity.append(response["main"]["humidity"])
            clouds.append(response["clouds"]["all"])
            wind.append(response["wind"]["speed"])
            lat.append(response["coord"]["lat"])
            lng.append(response["coord"]["lon"])
            cleaned_cities.append(city)
        except KeyError as e:
            print(str(response["cod"]) + str(e))
            print(response["message"])
            missing_cities.append(city)
        print("------------------")



1 of 19 cities called
coahuayana
------------------
2 of 19 cities called
pandan
------------------
3 of 19 cities called
hermanus
------------------
4 of 19 cities called
ulaangom
------------------
5 of 19 cities called
saint-pierre
------------------
6 of 19 cities called
punta arenas
------------------
7 of 19 cities called
jatai
------------------
8 of 19 cities called
nanortalik
------------------
9 of 19 cities called
georgetown
------------------
10 of 19 cities called
rio grande
------------------
11 of 19 cities called
dikson
------------------
12 of 19 cities called
mataura
------------------
13 of 19 cities called
dhidhdhoo
------------------
14 of 19 cities called
rikitea
------------------
15 of 19 cities called
barrow
------------------
16 of 19 cities called
ossora
------------------
17 of 19 cities called
maniitsoq
------------------
18 of 19 cities called
korla
404'main'
city not found
------------------
19 of 19 cities called
tsihombe
404'main'
city not found
-------

In [4]:
Weather_Dict = {
    "city": cleaned_cities,
    "temp":temp,
    "humidity":humidity,
    "clouds":clouds,
    "Wind Speed":wind,
    "lat":lat,
    "lng":lng,

}
Weather_DF = pd.DataFrame(Weather_Dict)
Weather_DF.tail()

,city,temp,humidity,clouds,Wind Speed,lat,lng
12,dhidhdhoo,81.11,100,20,9.19,6.88,73.10
13,rikitea,69.23,100,44,18.48,-23.12,-134.97
14,barrow,51.41,67,76,4.05,-38.31,-60.23
15,ossora,48.08,96,92,5.73,59.24,163.07
16,maniitsoq,33.95,100,76,10.31,65.42,-52.90


In [6]:
#Temp VS Latitude

city

'tsihombe'

In [7]:
#Humidity VS Latitude

In [8]:
#Cloudiness vs Latitude

In [9]:
#Wind Speed Vs Latitude

In [10]:
for lat in lats:
    print(sum(lats))

114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
114.72576645141228
